In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
from tqdm.notebook import tqdm

In [2]:
#Make sure to use GPU if available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True
print(device)

cuda:0


In [ ]:
#Load signals and labels
signals_mat = loadmat('signals_1.mat')['fringe'].transpose()
signals = np.concatenate((signals_mat.real,signals_mat.imag),axis=1)
labels_mat = loadmat('labels_1.mat')['label'].transpose()
labels = labels_mat[:,0]

print(signals.shape)
print(labels.shape)

(1000000, 256)
(1000000,)


In [ ]:
#Define training, test, and validation sets
x_train,x_test, y_train, y_test = train_test_split(signals, labels, train_size=0.8, stratify=labels)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, stratify=y_train)

print('Testing:')
print(x_test.shape)
print(y_test.shape)

print('Training:')
print(x_train.shape)
print(y_train.shape)

print('Validation:')
print(x_val.shape)
print(y_val.shape)


Testing:
(200000, 256)
(200000,)
Training:
(640000, 256)
(640000,)
Validation:
(160000, 256)
(160000,)


In [ ]:
#Scaling
sc = StandardScaler()
signals = sc.fit_transform(signals)

In [ ]:
#defining training, validation, and test classes
from torch.utils.data import Dataset, DataLoader
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length

trainset = dataset(x_train,y_train)
valset = dataset(x_val,y_val)
testset = dataset(x_test,y_test)

In [ ]:
#DataLoaders
trainloader = DataLoader(trainset,batch_size=512,shuffle=False)
valloader = DataLoader(valset,batch_size=512,shuffle=False)
testloader = DataLoader(testset,batch_size=512,shuffle=False)

In [ ]:
#defining the network
from torch import nn
from torch.nn import functional as F
class Net(nn.Module):
  def __init__(self,input_shape):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_shape,256)
    self.fc2 = nn.Linear(256,256)
    self.fc3 = nn.Linear(256,256)
    self.fc4 = nn.Linear(256,256)
    self.fc5 = nn.Linear(256,256)
    self.fc6 = nn.Linear(256,1)
    #self.fc7 = nn.Linear(256,1)
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.relu(self.fc3(x))
    x = torch.relu(self.fc4(x))
    x = torch.relu(self.fc5(x))
    x = torch.sigmoid(self.fc6(x))
    #x = torch.sigmoid(self.fc7(x))
    #x = torch.sigmoid(self.fc7(x))
    return x

In [ ]:
#hyper parameters
learning_rate = 0.01
epochs = 700

# Model , Optimizer, Loss
model = Net(input_shape=signals.shape[1])
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def fit(epochs, model, train_loader, val_loader, criterion, optimizer, scheduler):
    torch.cuda.empty_cache()
    losses = []
    accur = []

    fit_time = time.time()

    model = model.to(device)
    criterion = criterion.to(device)
    
    for e in range(epochs):

        epoch_loss = 0.0
        epoch_acc = 0.0

        #training loop
        model.train()
        for i, (x_train,y_train) in enumerate(tqdm(train_loader)):
            
            x_train = x_train.to(device)
            y_train = y_train.to(device)

            #forward pass
            output = model(x_train)
            loss = criterion(output, y_train.reshape(-1,1))

            #backward pass
            optimizer.zero_grad()
            loss.backward()

            #update weights
            optimizer.step()
 
 
        #validation loop
        model.eval()
        with torch.no_grad():

            for i, (x_val, y_val) in enumerate(tqdm(val_loader)):

                x_val = x_val.to(device)
                y_val = y_val.to(device)

                predicted = model(x_val)

                loss = criterion(predicted, y_val.reshape(-1,1))

                acc = np.mean(predicted.reshape(-1).detach().numpy().round() == y_val.numpy())

                epoch_loss += loss.item()
                epoch_acc += acc.item()

            print("epoch {}\tloss : {}\t accuracy : {}".format(e+1,epoch_loss / len(val_loader), epoch_acc / len(val_loader)))

In [ ]:
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, learning_rate, epochs=epochs,
                                            steps_per_epoch=len(trainloader))

print("Number of trainable parameters: {}".format(count_parameters(model)))
history = fit(epochs, model, trainloader, valloader, loss_fn, optimizer, sched)

Number of trainable parameters: 329217


  0%|          | 0/1250 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

epoch 0	loss : 0.6926029196943337	 accuracy : 0.528660393370607


  0%|          | 0/1250 [00:00<?, ?it/s]

...

epoch 2	loss : 0.6909897491193047	 accuracy : 0.5596171126198083


  0%|          | 0/1250 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

epoch 3	loss : 0.6893496233434342	 accuracy : 0.5707805011980831


  0%|          | 0/1250 [00:00<?, ?it/s]